In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

# Load the Hitters dataset from ISLR
Hitters = pd.read_csv('https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/ISLR/Hitters.csv')
Hitters.drop(['rownames'],axis = 1, inplace = True)

In [3]:
# Remove rows with missing values
Hitters = Hitters.dropna()

# Convert categorical variables to dummy/indicator variables
Hitters = pd.get_dummies(Hitters, columns=['League', 'Division', 'NewLeague'], drop_first=False)

# Ensure all data is numeric
Hitters = Hitters.apply(pd.to_numeric, errors='coerce')

# Split the data into training (70%) and test (30%) sets
train_data, test_data = train_test_split(Hitters, test_size=0.3, random_state=123)

In [4]:
# Define the control for cross-validation on the training data
X_train = train_data.drop(['Salary'], axis=1)
y_train = train_data['Salary']
X_test = test_data.drop(['Salary'], axis=1)
y_test = test_data['Salary']


In [6]:
# Perform best subset selection on training data using SequentialFeatureSelector
lr = LinearRegression()

# Use cross-validation to find the optimal number of features
cv_results = []
for k in range(1, len(X_train.columns)):
    sfs = SequentialFeatureSelector(lr, n_features_to_select=k, direction='forward', cv=10)
    sfs.fit(X_train, y_train)
    score = np.mean(cross_val_score(lr, X_train.iloc[:, sfs.get_support()], y_train, cv=10, scoring='neg_mean_squared_error'))
    cv_results.append((k, score))

print(cv_results)

# Find the best number of features
best_k = sorted(cv_results, key=lambda x: x[1], reverse=True)[0][0]

# Fit the final model using the selected features
sfs = SequentialFeatureSelector(lr, n_features_to_select=best_k, direction='forward', cv=10)
sfs.fit(X_train, y_train)
selected_features = sfs.get_support(indices=True)
selected_feature_names = X_train.columns[selected_features]
print("Selected Features:", selected_feature_names)

[(1, np.float64(-140159.13680461078)), (2, np.float64(-121615.29093729786)), (3, np.float64(-114950.32217226736)), (4, np.float64(-115562.9863783058)), (5, np.float64(-115674.8485244296)), (6, np.float64(-116164.24583808996)), (7, np.float64(-116145.09963585413)), (8, np.float64(-116361.95234133741)), (9, np.float64(-112761.19131112802)), (10, np.float64(-112761.19131112797)), (11, np.float64(-113725.65952755357)), (12, np.float64(-113725.65952755362)), (13, np.float64(-113900.5712909582)), (14, np.float64(-113900.57129095818)), (15, np.float64(-114181.20034588668)), (16, np.float64(-115508.60332116422)), (17, np.float64(-116423.44012302443)), (18, np.float64(-118218.33515094586)), (19, np.float64(-121035.65558803562)), (20, np.float64(-122260.63175427246)), (21, np.float64(-126368.11707344881))]
Selected Features: Index(['Hits', 'Runs', 'Walks', 'Years', 'CRuns', 'CWalks', 'PutOuts', 'Errors', 'Division_E', 'Division_W'], dtype='object')


In [7]:
# Train the final model using the selected features
final_model = LinearRegression()
final_model.fit(X_train.iloc[:, selected_features], y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [ ]:
# Calculate the training RMSE for the cross-validated model
train_predictions = final_model.predict(X_train.iloc[:, selected_features])
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
print(f"Training RMSE: {train_rmse}")

In [ ]:
# Evaluate the cross-validated model on the test data
test_predictions = final_model.predict(X_test.iloc[:, selected_features])
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_r2 = r2_score(y_test, test_predictions)

print(f"Cross-validated Test RMSE: {test_rmse}")
print(f"Cross-validated Test R-squared: {test_r2}")

In [ ]:
# Convert boolean columns to integers (1/0)
X_train_selected = X_train_selected.applymap(lambda x: int(x) if isinstance(x, bool) else x)

In [ ]:
# Fit the final model using statsmodels to evaluate the significance of the coefficients
ols_model = OLS(y_train, X_train_selected).fit()

# Display the summary of the model
print(ols_model.summary())